<p id="part0"></p>
​
<p style="font-family: Arials; line-height: 2; font-size: 44px; font-weight: bold; letter-spacing: 0px; text-align: center; color: #FF8C00">This program, first crops dicom images, and then turns them into png images</p>
​
<img src="https://th.bing.com/th/id/OIP.ZZ5i8bsrLufsWzXxAxYpDwHaD4?pid=ImgDet&rs=1" width="90%" align="center" hspace="20%" vspace="5%"/>
​
<p style="font-family: Arials; font-size: 40px; font-style: normal; font-weight: bold; letter-spacing: -2px; color: #000000; line-height:2.0">Content:</p>
​
<p style="font-family: Arials; font-size: 18px; font-style: normal; font-weight: bold; letter-spacing: 0px; color: #000000; line-height:.5"><a href="#part1" style="color:#000000"> 1- Importing libraries</a></p>

<p style="font-family: Arials; font-size: 18px; font-style: normal; font-weight: bold; letter-spacing: 0px; color: #000000; line-height:.5"><a href="#part2" style="color:#000000"> 2-Describing dicom files!</a></p>

<p style="text-indent: 1vw; font-family: Arials; font-size: 16px; font-style: normal; font-weight: bold; letter-spacing: 0px; color: #000000; line-height:0.5">
<a href="#part2-1" style="color:#000000">  2-1 Reading dicom files</a></p>

<p style="text-indent: 1vw; font-family: Arials; font-size: 16px; font-style: normal; font-weight: bold; letter-spacing: 0px; color: #000000; line-height:0.5">
<a href="#part2-2" style="color:#000000">2-2 Cropping dicom files </a></p>

<p style="text-indent: 1vw; font-family: Arials; font-size: 16px; font-style: normal; font-weight: bold; letter-spacing: 0px; color: #000000; line-height:0.5">
<a href="#part2-3" style="color:#000000">2-3 Comparing original images with cropped ones </a></p>

<p style="text-indent: 1vw; font-family: Arials; font-size: 16px; font-style: normal; font-weight: bold; letter-spacing: 0px; color: #000000; line-height:0.5">
<a href="#part2-4" style="color:#000000">2-4 Turning dicom images to png ones </a></p>


<p id="part1"></p>

<p style="font-family: Times New Roman; font-size: 20px; font-style: bold; font-weight: bold; letter-spacing: 0px; color: #0000FF">1- Importing Libraries<p>
<hr style="height: 1px; border: 1; background-color: #0000FF">

In [ ]:
# For better performance in turning dicom to png files We uysed this command
!pip install -qU python-gdcm pydicom pylibjpeg 

In [ ]:
import pandas as pd
import numpy as np
import pydicom, cv2,glob,os,time,gdcm
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

<p id="part2"></p>

<p style="font-family: Times New Roman; font-size: 20px; font-style: bold; font-weight: bold; letter-spacing: 0px; color: #0000FF">2- Describing dicom files!</p>
<hr style="height: 1px; border: 1; background-color: #0000FF">



**DICOM** is an acronym for Digital Imaging and Communications in Medicine. Files in this format are most likely saved with either a DCM or DCM30 (DICOM 3.0) file extension, but some may not have an extension at all.

DICOM is both a communications protocol and a file format, which means it can store medical information, such as ultrasound and MRI images, along with a patient's information, all in one file. The format ensures that all the data stays together, as well as provides the ability to transfer said information between devices that support the DICOM format.

**What package is used for this kind of file in python?**
pydicom is the library you can use in order to open dcm files and manupulate them.Actually, pydicom is a pure Python package for working with DICOM files such as medical images, reports, and radiotherapy objects. For more information , click this [link](https://pydicom.github.io/)

<p id="part2-1"></p>

<p style="font-family: Times New Roman; font-size: 20px; font-style: bold; font-weight: bold; letter-spacing: 0px; color: #0000FF">2-1 Reading dicom files</p>
<hr style="height: 1px; border: 1; background-color: #0000FF">

# Before running the following code, please add rsna-breast-cancer dataset by clicking + on the right page

In [ ]:
dataset_dir= '/kaggle/input/rsna-breast-cancer-detection/'
train_images_dir = sorted(glob.glob(f'{dataset_dir}train_images/*/*.dcm'))

number_of_iamges = len(train_images_dir)
print('Number of train images is:',number_of_iamges) # You are xepceted to see:54706

In [ ]:

def read_dicom(dicom_file):
    '''This function takes image with dicom format and reads it, then returns image's pixel as an output'''
    
    dicom_reader = pydicom.dcmread(dicom_file) # it reads dciom file
    dicom_pixel = dicom_reader.pixel_array # it collects the pixel

    dicom_pixel = (dicom_pixel - dicom_pixel.min()) / (dicom_pixel.max() - dicom_pixel.min()) # Normalization process

    if dicom_reader.PhotometricInterpretation == "MONOCHROME1":
        dicom_pixel = 1 - dicom_pixel # a special kind of format in dicom files, you can search it on the net
    
   
    return dicom_pixel

<p id="part2-2"></p>

<p style="font-family: Times New Roman; font-size: 20px; font-style: bold; font-weight: bold; letter-spacing: 0px; color: #0000FF">2-2 Crop dicom file</p>
<hr style="height: 1px; border: 1; background-color: #0000FF">

Define a function to crop dicom file, sine our images have lots of unuseful pixels, using cv2.connectedComponentsWithStats

check this [link](https://stackoverflow.com/questions/35854197/how-to-use-opencvs-connectedcomponentswithstats-in-python) for more detail

In [ ]:
def crop_dicom(dicom_file):
    '''This function takes dicom image and removes additional space/pixels of it based on the logic behind connectedComponentsWithStats method
    ,then it returns a cropped dicom file'''
    
    result= cv2.connectedComponentsWithStats((dicom_file > 0.005).astype(np.uint8)[:, :], 8, cv2.CV_32S)
    # result has 4 diffrenet outputs. You can see different outputs here.
    # But for reconizing the black color(0) from others(1), we just need result[2] , called stat matrix
    
    stat_matrix = result[2] # include left, top, width, height, area_size columns
    # the second row shows the boxes of pixels which are different from background.Also, we don't need are_size cloumn. So:
    
    second_row = stat_matrix[1:, 4].argmax() + 1
    x1, y1, w, h = stat_matrix[second_row][:4]
    x2 = x1 + w
    y2 = y1 + h
    cropped_dicom = dicom_file[y1: y2, x1: x2]
    
    return cropped_dicom

<p id="part2-3"></p>

<p style="font-family: Times New Roman; font-size: 20px; font-style: bold; font-weight: bold; letter-spacing: 0px; color: #0000FF">2-3 Comparing original images with cropped ones</p>
<hr style="height: 1px; border: 1; background-color: #0000FF">

In [ ]:
test_images = train_images_dir[:2]

for image in test_images:
    patient_id = image.split('/')[-2] # split the patient_id from the whole address
    image_id = image.split('/')[-1][:-4] # split the image_id from the whole address
    

    # Plot Orginial iamge
    plt.figure(figsize=(8,8))
    dicom_pixel = read_dicom(dicom_file=image)
    plt.imshow(dicom_pixel,cmap='gray')
    plt.title(f"Original dicom image-->image_id:{image_id}",fontsize=16)
    plt.show()
    
    # Plot cropped image
    plt.figure(figsize=(8,8))
    cropped_iamge = crop_dicom(dicom_file=dicom_pixel)
    plt.imshow(cropped_iamge,cmap='gray')
    plt.title(f"Cropped dicom image-->image_id:{image_id}",fontsize=16)
    plt.show()

<p id="part2-4"></p>

<p style="font-family: Times New Roman; font-size: 20px; font-style: bold; font-weight: bold; letter-spacing: 0px; color: #0000FF">2-4 Turning dicom images  into png images</p>
<hr style="height: 1px; border: 1; background-color: #0000FF">

In [ ]:
# Creating two folders called 'Cancer' and 'No_Cancer' for categorizing images into two groups.

SAVE_FOLDER_CANCER = "/kaggle/working/Cancer/"
os.makedirs(SAVE_FOLDER_CANCER, exist_ok=True)

SAVE_FOLDER_NO_CANCER = "/kaggle/working/NO_Cancer/"
os.makedirs(SAVE_FOLDER_NO_CANCER, exist_ok=True)

In [ ]:
# In order to turn the dicom images into png files, firstly we need to have train_csv file to match images with the corresponding info.
train_df_dir = "/kaggle/input/rsna-breast-cancer-detection/train.csv"
train_df = pd.read_csv(train_df_dir)
train_df[['patient_id','image_id','site_id','machine_id']] = train_df[['patient_id','image_id','site_id','machine_id']].astype(str)

# Creating the function
def dcm_to_png(file,size=256,save_folder="/kaggle/working/", extension ='png'):
    '''This function takes the dicom file, reads it using read_dicom function, crops it using crop_dicom function, then resizes it to 512x512 
    and then turns it to png format. Finally it saves images inside two created folders'''
    
    patient_id = file.split('/')[-2]
    image_id = file.split('/')[-1][:-4]
    dicom_pixel = read_dicom(dicom_file=file) 
    cropped_dicom = crop_dicom(dicom_file=dicom_pixel) 
    resized_img = cv2.resize(cropped_dicom,dsize=(size,size)) 
    
    loc_in_train_df = np.where(train_df.image_id==image_id)
    id_cancer = train_df.iloc[loc_in_train_df].cancer
    if id_cancer.all():
        cv2.imwrite(save_folder +"Cancer/" + f"{patient_id}_{image_id}.{extension}", (resized_img*255).astype(np.uint8))
    else:
        cv2.imwrite(save_folder +"NO_Cancer/" + f"{patient_id}_{image_id}.{extension}", (resized_img*255).astype(np.uint8))

In [ ]:
# Calling the dcm_to_png function
# if you want to test just some files, please use train_images[:10] instead of the whole train_images_dir

# Using parallel function to make process faster
start = time.time()
_ = Parallel(n_jobs=4)(
    delayed(dcm_to_png)(file, size=256)
    for file in tqdm(train_images_dir[:30000])  # you can use train_images for all iamges
)
stop = time.time()
print(f'This process took you:{stop-start} seconds')
print('Now, you can have access to cropped files wtih png format')